### Machine Learning Project: English Grapheme to Phoneme Conversion



#### Introduction
For my project, I chose to tackle the problem of converting graphemes to phonemes in English words. English is known for being fairly inconsistent with its spelling. Words with similarities in their orthographical representations (like 'cough' and 'though') often have very different pronunciations. English is also quite heavy with homographs (like the verb 'lead' and the noun 'lead' that describes a type of metal) which have identical orthographical representations but different pronunciations. Considering these aspects of English spelling, it can be very difficult (or sometimes impossible) for a non-native English speaker to predict how a word they have never seen before is correctly pronounced. For this reason, my initial expectations for a computer trying to succeed in this task aren't very high. It is also the reason why I find this task interesting: given these limiting factors, how well can English pronunciation be predicted utilizing machine learning methods?

#### Data

The data I will be using for this task is gathered from the CMU dictionary entries. It is an English spelling dictionary, which consists of pairs of words and their pronunciations. It can be accessed in the NLTK corpus package. I will start by importing the cmudict data as well as the rest of NLTK which I will need in the later parts of this task.

In [ ]:
import nltk
from nltk.corpus import cmudict

My approach to predicting the pronunciations of words is based on the assumption that graphemes tend to represent individual phonemes. However, there is already a problem in this approach, as this is obviously not always the case. How do we deal with words which have a different number of graphemes and phonemes? For now, I will simply avoid this problem by only picking the words in the dictionary for which the number is the same for both, and see how good results I can achieve for this limited dataset. However, I will return to the problem in a later section of this report. Also, for the sake of simplicity, I will not try to predict word stress as a part of this task, so the numeric values that represent the stress will be removed using regular expression operations.

The cmudict entries are formatted as lists of tuples. In the first index of the tuple is a word (string) and in the second index is the pronunciation for the word as a list of phonemes (strings). Since my approach involves the assumption that graphemes correspond to single phonemes, a logical format for the data is pairs of graphemes and phonemes. This is analogical to word-tag pairs used in tasks involving POS tagging, so using this format will ensure that it is be possible to utilize the taggers included in NLTK to this task as well.

First, using a list comprehension, I will create a list of tuples with two lists inside each tuple. The first list inside the tuple will consist of the graphemes (represented by the variable "g" in the list comprehension) of a word and the second list will consist of the phonemes (represented by "p") with the numeric values indicating stress removed. Next, I will zip the two lists inside each tuple to create a list of lists with tuples inside. So, our final data is essentially a list of words formatted as lists of grapheme-phoneme pairs.

In [ ]:
import re

data = [([g for g in word], [re.sub('\d', '', p) for p in pronunciation]) 
        for (word, pronunciation) in cmudict.entries() if len(word) == len(pronunciation)]

paired_data = [list(zip(g, p)) for (g, p) in data]

The next step is to divide the data to train and test sets. I will be using a 90-10 split for this task, picking every 10th item from the processed data for the test set using extended slicing and incorporating the rest of the data in the train set. I will define a function for this part to avoid repetition as I intend to repeat a similar process in a later section of this project.

In [ ]:
def get_sets(data):
    test_set = data[::10]
    train_set = [pair for pair in data if pair not in test_set]

    return train_set, test_set

train_set, test_set = get_sets(paired_data)

#### Methods

I chose to utilize the Hidden Markov Model for this task. It appears the best fit out of all the supervised machine learning methods introduced in this course due to its capabilities to account for the overall probability of a phoneme for a given grapheme as well as the effect that the previous phoneme has on the one being predicted. It should also be noted that one crucial shortcoming of HMM, its inability to handle out-of-vocabulary words (or graphemes, in this case), does not affect this task as it is extremely unlikely that there are graphemes in our test set that do not appear in the train set. I will start by importing the HMM module from NLTK. Then, I will create a HMM trainer and use it to train the tagger with the train set data defined in the previous section.

In [ ]:
from nltk import hmm

trainer = hmm.HiddenMarkovModelTrainer()
tagger = trainer.train_supervised(train_set)

#### Results & Discussion

With the HMM tagger ready, we can evaluate its performance on the test set:

In [ ]:
print("The accuracy of the HMM tagger:", tagger.evaluate(test_set))

Considering the anticipated difficulty of this task, this tagger performs surprisingly well, tagging approximately 75 percent of all graphemes with the correct phonemes. However, considering that the tagger gets a fourth of its predictions wrong, we should be able to identify some consistent errors and their causes by observing some of the tagged words. To visualize the performance of the tagger, I will have the program print some of the tagged words alongside their correct pronunciation.

In [ ]:
for w in test_set[::200]:
    untagged = nltk.untag(w)
    tagged = tagger.tag(untagged)
    print("Word:", "".join([g for g, p in tagged]))
    print("Predicted pronunciation:", "-".join([p for g, p in tagged]))
    print("Correct pronunciation:  ", "-".join([p for g, p in w]))
    i = 0
    correct = 0
    for g, p in tagged:
        if p == w[i][1]:
            correct += 1
        i += 1
    print("Accuracy: " + str(correct) + "/" + str(len(tagged)) + " phonemes (" 
          + str(round(correct / len(tagged) * 100, 2)) + "%)")
    print()

These example results demonstrate some shortcomings of the HMM tagger in this task specifically. These shortcomings have to do with word boundaries and three-letter sequences that appear commonly in English. Comparing the predicted and correct pronuciations for the word "likable", we can see that the prediction is fairly inaccurate. This is largely due to the fact that the model cannot predict the correct pronunciation for the very common word-final letter sequence of "ble", because it doesn't know where the word will end (as it processes the letters sequentially from the beginning until it runs out of letters) and when tagging a letter with a phoneme, it only looks at the phoneme that it assigned to the letter before the current one, not any letters before that. Therefore, it sees the letter b represented by the phoneme B followed by the letter l and assigns the most common phoneme for this two-phoneme sequence, resulting in the phoneme sequence of B-L, which is a very common word initial consonant cluster but unlikely to appear later in the word.

Another matter worth noting is the lack of consideration for word stress. As I have mentioned before, I chose to not incorporate it in this model for the sake of simplicity. However, it is clear that word stress plays an important role in pronunciation of the English language. Especially vowels are affected by stress: there is a strong tendency for any vowel to be reduced to schwa in unstressed syllables. This is demonstrated in the example word "contaminated". Here, the first syllable is unstressed and thus the vowel is reduced to schwa (AH). The model makes its predictions without knowing where the word stress is placed, so it chooses the more frequent phoneme OW for the grapheme o. To achieve accurate results in these scenarios, one would have to consider the effect of stress in the model.

To visualize the performance of the tagger and to demonstrate the shortcomings discussed, I will create a confusion matrix of the results. It allows for a further examination of which phonemes are tagged correctly and which are not. First, I will need to extract a list which consist of only the phonemes from the correctly tagged test set (the gold standard) as well as from the test set tagged by the HMM tagger. These operations will be done by using slightly altered code from the fifth chapter of the NLTK book. For the sake of readability, the matrix will only include the 15 most common phonemes in the test set.

In [ ]:
def list_phonemes(tagged_words):
    return [phoneme for word in tagged_words for (letter, phoneme) in word]

def apply_tagger(tagger, data):
    return [tagger.tag(nltk.untag(word)) for word in data]

gold = list_phonemes(test_set)
test = list_phonemes(apply_tagger(tagger, test_set))

cm = nltk.ConfusionMatrix(gold, test)
print(cm.pretty_format(sort_by_count=True, show_percents=True, truncate=15))

Looking at the confusion matrix, it appears that the model struggles the most with differentiating vowels from each other. This makes sense, as it reflects the properties of English spelling: it is much more common for multiple vowels to be represented by a single grapheme than it is for consonants. It is also the result of the matters regarding vowel quality discussed earlier.

#### Comparative Analysis

The results I have obtained so far do have some issues. As I mentioned before, the data used to training and testing only includes words for which the number of graphemes and phonemes is the same. If we take a look at the whole corpus from which our datasets were extracted from, we will see that the datasets might not exactly be representative of the English language:

In [ ]:
print(str(round((len(train_set)+len(test_set))/len(cmudict.entries())*100, 1)) + "%")

Only about one fourth of words in the CMU dictionary have the same amount of graphemes and phonemes. This means that the majority of English words do not follow this pattern, which raises the question: how well can we train a model to predict the pronunciation for all English words? To begin to solve this problem with the methods I am using, another issue needs to be solved. As the tagger will always tag a single grapheme with a single phoneme, the lengths of the grapheme sequences and the phoneme sequences need to match. One possible way of solving this problem is to modify the "non-matching" word-pronunciation pairs by augmenting the shorter element of the pair with padding. However, it is unclear where on the word or pronunciation the padding should go. Because of this, I will try two different approaches, padding at the end of the sequence and at the beginning.

I will start with adding the padding to the end of the shorter sequence. I will use the acronym "PAE" (Padding At the End) to mark all parts of my program that have to do with this task specifically. First, I need to create a new dataset with all of the CMU dictionary entries included.

In [ ]:
data2 = [([g for g in word], [re.sub('\d', '', p) for p in pronunciation]) 
            for (word, pronunciation) in cmudict.entries()]

Then, I will go through the dataset to find the word-pronunciation pairs with different lengths and add the padding at the end of the shorter element with a for-loop, appending a "PAD" item to the shorter list of elements (graphemes or phonemes) until the lists are of equal length. I will use the function defined earlier to extract new train and test sets from the modified data.

In [ ]:
for (word, pronunciation) in data2:
    if len(word) > len(pronunciation):
        for i in range(len(word) - len(pronunciation)):
            pronunciation.append("<PAD>")
    elif len(pronunciation) > len(word):
        for i in range(len(pronunciation) - len(word)):
            word.append("<PAD>")
            
paired_data_pae = [list(zip(g, p)) for (g, p) in data2]

train_pae, test_pae = get_sets(paired_data_pae)

tagger_pae = trainer.train_supervised(train_pae)

Now, I will repeat similar procedures to add the padding to the beginning of the shorter sequences in the pairs. Similarly to the previous part, I will use "PAB" (Padding At the Beginning) to mark the variables used for this part. Again, this will be done with a for-loop, this time inserting the padding to the first index of the list until the lengths of the two lists match.

In [ ]:
data2 = [([g for g in word], [re.sub('\d', '', p) for p in pronunciation]) 
            for (word, pronunciation) in cmudict.entries()]

for (word, pronunciation) in data2:
    if len(word) > len(pronunciation):
        for i in range(len(word) - len(pronunciation)):
            pronunciation.insert(0, "<PAD>")
    elif len(pronunciation) > len(word):
        for i in range(len(pronunciation) - len(word)):
            word.insert(0, "<PAD>")

paired_data_pab = [list(zip(g, p)) for (g, p) in data2]

train_pab, test_pab = get_sets(paired_data_pab)

tagger_pab = trainer.train_supervised(train_pab)

Having trained two new taggers, we can now take a look at how they perform on their corresponding test sets:

In [ ]:
print("Accuracy of the HMM tagger (all cmudict entries, padding at the end):", tagger_pae.evaluate(test_pae))
print("Accuracy of the HMM tagger (all cmudict entries, padding at the beginning):", tagger_pab.evaluate(test_pab))

These taggers seem to perform quite poorly. It appears that padding the shorter elements is not a sufficient approach to this problem. Out of the two new taggers I have trained, the tagger trained with the data that has padding at the end of the sequences is the better performer by a fair margin. This would indicate that in English, it is more common for "silent letters" (graphemes in a word that do not correspond with any phonemes in the pronunciation) to appear at the end of the word rather than at the beginning.

However, another question remains: what kind of results could be achieved if these taggers that are trained with larger sets of words were applied to the "easier" test sets, those from the earlier part of this project with only words with an equal number of graphemes and phonemes in them? Presumably, the taggers would have an easier time with this task. I will try this for both of the new taggers and compare the performances with the performance of the original tagger from the previous section.

In [ ]:
print("Accuracy (train: all entries, test: n_graph = n_phon, pad at the end):", tagger_pae.evaluate(test_set))
print("Accuracy (train: all entries, test: n_graph = n_phon, pad at the beginning):", tagger_pab.evaluate(test_set))
print("Accuracy (train: n_graph = n_phon, test: n_graph = n_phon)", tagger.evaluate(test_set))

Again, the two new taggers show poor performance. This should probably be expected: the new taggers were trained with data that looks quite different from the data on which their performance is tested. Accordingly, the original tagger is well suited to handle the tagging for this test set, as it is very similar to the set it was trained on. However, the assumption that the new taggers would perform better for this "easier" test set than for the test set extracted from all of the cmudict entries turned out to be correct. The accuracy for this test set is about 10 percent higher for both of the new taggers.

#### Conclusion

In this project, my task was train a machine learning model to predict which graphemes convert to which phonemes in English words found in the CMU pronunciation dictionary. I simplified the approach to this task in two main ways. Firstly, the main focus of this project was on English words that have the same number of graphemes and phonemes. Secondly, I did not include predicting stress as a part of this project. These limitations were taken into account in my evaluation of the results.

I found that my approach using a simple HMM tagger yielded fairly accurate results for the first part of my task, that is, for words which have the same number of graphemes and phonemes. When two new HMM taggers were trained with the entirety of the CMU dictionary, the results turned out poor. This is the result of several factors, including the limitations of the fairly simple tagger and the inconsistent properties of English spelling that make the task of predicting pronunciation based on orthographical representation quite difficult.

My approach to solving the task of grapheme-to-phoneme conversion leaves much to be desired. Preferably, a more complex machine learning model should be used for this task, one capable of taking into account aspects of language such as word stress, word boundaries and common affixes. However, I believe that a simple model such as the one used in this project would be quite sufficient for a language such as Finnish, as its pronunciation tends to stay rather true to its spelling.